In [1]:
import sys
import torch
import numpy as np
import random
import os
import time
import pdb
from models import create_Inet
import torch
import torchvision
import torchvision.transforms as transforms
from scipy import interpolate
from utils.image_folder import ImageFolder
from utils import utils
from utils.utils import *
# from utils.Visualizer import Visualizer
import matplotlib.pyplot as plt
from IPython.display import clear_output
from options import DefaultConfig
opt = DefaultConfig()
torch.cuda.set_device(opt.gpu_ids)
torch.manual_seed(opt.seed)
torch.cuda.manual_seed_all(opt.seed)
np.random.seed(opt.seed)
random.seed(opt.seed)
torch.backends.cudnn.deterministic = True

Interaction Network: initialized


In [2]:
opt.dataroot_train = opt.dataroot + 'train/'
dataset_train = ImageFolder(opt.dataroot_train,
                       transform=transforms.Compose([
                                   transforms.Resize(opt.loadSize),
                                   transforms.RandomResizedCrop(opt.fineSize),
                                   transforms.RandomHorizontalFlip(),
                                   transforms.ToTensor()]))
dataset_loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=opt.batch_size, shuffle=True)
dataset_train_size = len(dataset_train)
print('#training images = %d' % dataset_train_size)

opt.dataroot_val = opt.dataroot + 'subval/'
dataset_val = ImageFolder(opt.dataroot_val,
                       transform=transforms.Compose([
                                   transforms.Resize(opt.loadSize),
                                   transforms.RandomResizedCrop(opt.fineSize),
                                   transforms.ToTensor()]))
dataset_loader_val = torch.utils.data.DataLoader(dataset_val, batch_size=opt.batch_size_val, shuffle=True)
dataset_val_size = len(dataset_val)
print('#training images = %d' % dataset_val_size)

#training images = 118060
#training images = 1000


In [3]:
model = create_Inet(opt)
model.setup(opt)
# print(model)
if opt.gpu_ids is not None:
    model.cuda()

Interaction net was created


In [4]:
!nvidia-smi 

Sun Apr  5 02:22:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 440.59       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:19:00.0 Off |                  N/A |
| 18%   24C    P8     9W / 250W |     11MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  On   | 00000000:1A:00.0 Off |                  N/A |
| 18%   32C    P2    53W / 250W |    906MiB / 11019MiB |      6%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [5]:
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

2020-04-05 02:22:20


Code optimisation:
1. rewrite sampling code using numpy
2. rewrite network structure and make it compataple for non-click input
3. rewrite validation code 

In [6]:
X_val = []
X_train = []

val_losses = []
train_losses = []
val_psnrs = []
train_psnrs = []

total_steps = 0
for epoch in range(opt.epoch_count, opt.niter + opt.niter_decay):
    val_step = 0
    model_index = 0
    for i, data_raw in enumerate(dataset_loader_train):
        data_np = utils.get_colorization_data(data_raw, opt, p=opt.sample_p)
        data = ToCuda(data_np)
        print('x'+1)
        total_steps += 1
        val_step += 1

        em_ab = model(data['gray'], data['clicks'], data['prev'])
        model.optimizer.zero_grad()
        loss = model.criterion(em_ab, data['ab'])
        loss.backward()
        model.optimizer.step()

        if val_step % opt.val_freq == 0:
            print('validation phase')
            val_loss = 0
            val_psnr = 0

            model.eval()
            for ii, data_raw_val in enumerate(dataset_loader_val):
                data_np_val = utils.get_colorization_data(data_raw_val, opt, p=opt.sample_p)
                data_val = ToCuda(data_np_val)
                em_ab_val = model(data_val['gray'], data_val['clicks'], data_val['prev'])

                scores = model.criterion(em_ab_val, data_val['ab'])
                em_ab_np_val = em_ab_val.detach().cpu().numpy()
                psnr = PSNR(data_np_val, em_ab_np_val, opt, avg=False)

                val_loss += scores.detach().cpu().numpy()
                val_psnr += psnr
            val_psnrs.append(val_psnr / dataset_val_size)
            val_losses.append(val_loss / dataset_val_size)
            X_val.append(total_steps)
            model.train()
            
            # save model
            model_index += 1
            save_model(model, opt, epoch, model_index, val_psnr / dataset_val_size)

        if total_steps % opt.print_freq == 0:
            X_train.append(total_steps)
            # loss and PSNR
            em_ab_np = em_ab.detach().cpu().numpy()
            train_psnrs.append(PSNR(data_np, em_ab_np, opt))
            train_losses.append(loss.detach().cpu().numpy())
            
            # plot loss and PSNR curve
            clear_output(wait=True)
            plt.figure(1)
            plt.rcParams['figure.dpi'] = 120 
            plt.ylim([0, .03])
            plt.title('Loss')
            plt.plot(X_train, train_losses, label="Training loss")
            plt.plot(X_val, val_losses, label="Validation loss")
            plt.grid(True)
            plt.legend()
            plt.show()

            plt.figure(2)
            plt.rcParams['figure.dpi'] = 120 
            plt.ylim([-30, 30])
            plt.title('PSNR')
            plt.plot(X_train, train_psnrs, label="Training PSNR")
            plt.plot(X_val, val_psnrs, label="Validation PSNR")
            plt.grid(True)
            plt.legend()
            plt.show()

    np.save('Iter'+str(epoch)+'train_loss.npy', train_losses)
    np.save('Iter'+str(epoch)+'val_loss.npy', val_losses)
    np.save('Iter'+str(epoch)+'train_psnr.npy', train_psnrs)
    np.save('Iter'+str(epoch)+'val_psnrs.npy', val_psnrs)


TypeError: must be str, not int

In [ ]:
#!sudo apt-get install -y psmisc
#!fuser /dev/nvidia*

In [ ]:
#!kill 1529

In [8]:
data_np

{'gray': tensor([[[[-0.1879, -0.1835, -0.1793,  ..., -0.0962, -0.0991, -0.0991],
           [-0.1848, -0.1879, -0.1793,  ..., -0.0860, -0.0860, -0.0937],
           [-0.1894, -0.1922, -0.1833,  ..., -0.0816, -0.0778, -0.0857],
           ...,
           [-0.2495, -0.2380, -0.2250,  ..., -0.3321, -0.3131, -0.2748],
           [-0.2362, -0.2338, -0.2191,  ..., -0.2950, -0.3215, -0.3112],
           [-0.2305, -0.2427, -0.2103,  ..., -0.2879, -0.2967, -0.2880]]],
 
 
         [[[-0.1412, -0.1155, -0.0947,  ..., -0.0875, -0.0935, -0.0935],
           [-0.1424, -0.1241, -0.1104,  ..., -0.0834, -0.0899, -0.0931],
           [-0.1499, -0.1286, -0.1185,  ..., -0.0793, -0.0870, -0.0678],
           ...,
           [-0.4664, -0.4680, -0.4642,  ..., -0.0896, -0.0763, -0.0978],
           [-0.4664, -0.4721, -0.4694,  ..., -0.0937, -0.0915, -0.0953],
           [-0.4647, -0.4716, -0.4694,  ..., -0.0949, -0.0813, -0.0782]]],
 
 
         [[[-0.3543, -0.3452, -0.3400,  ..., -0.3431, -0.3413, -0.3368],

In [ ]:
data_np

In [ ]:
em_img = np.append(data_np['gray'][2,:,:,:], em_ab_np[2,:,:,:], axis=0) * opt.norm_factor
gt_img = np.append(data_np['gray'][2,:,:,:], data_np['ab'][2,:,:,:], axis=0) * opt.norm_factor

In [ ]:
import skimage
gt_rgb = skimage.color.lab2rgb(np.transpose(gt_img, (1, 2, 0)))
skimage.io.imshow(gt_rgb)